In [ ]:
import pandas as pd
import numpy as np
import ast

In [ ]:
# file_path="/Users/gyauk/github/Project1_movie_analysis/Project1/data/raw/movies_07-04-2025_16-31-38.csv"
file_path='/Users/gyauk/github/Project1_movie_analysis/Project1/data/raw/movies_08-04-2025_10-27-01.csv'

df=pd.read_csv(file_path)
df.head()

In [ ]:
df['status']

In [ ]:
df.columns.to_list()


In [ ]:
df['belongs_to_collection'][0]

In [ ]:
# df['credits'][0]


## Data Preparation & Cleaning

### Drop Irrelevant Columns

In [ ]:
df= df.drop(columns =['adult', 'imdb_id', 'original_title', 'video', 'homepage'])
df.head()

In [ ]:
print(df['belongs_to_collection'][5])
df['production_countries'][10]


In [ ]:
print(type(df['production_countries'][0]))

for col in df.columns:
    print(f"{col}: {type(df[col].iloc[0])}")


# x=df['belongs_to_collection'][5]

# type(x)

### **Evaluate JSON Like column**

In [ ]:
import ast

def evaluate_json_column(column):
    try:
        return ast.literal_eval(column) if pd.notna(column) else {}
    except (ValueError, SyntaxError):
        return {}

json_columns = ['belongs_to_collection', 'genres', 'production_countries', 
                'production_companies', 'spoken_languages','credits']

for col in json_columns:
    df[col] = df[col].apply(evaluate_json_column)


df.head()

In [ ]:
type(df['production_countries'][0])

In [ ]:
type(df['credits'][0])


In [ ]:
x=df['belongs_to_collection'][5]

type(x)

In [ ]:
df['genres'][5]

In [ ]:
for col in df.columns:
    print(f"{col}: {type(df[col].iloc[0])}")

### **Extract and clean key data points**

In [ ]:
def extract_collection_name(value):
    try:
        if pd.notnull(value) and isinstance(value, dict):
            return value.get('name')
    except (ValueError, SyntaxError):
        return None
    # return None

# Apply to the column
df['collection_name'] = df['belongs_to_collection'].apply(extract_collection_name)


In [ ]:
df['collection_name'][0]

In [ ]:
def break_data_points(df, init_column='genres', new_column='genre_names'):
  df[new_column] = df[init_column].apply(lambda x: ' | '.join(d['name'] for d in x) 
                  if isinstance(x, list) else None)
  return df
      
    #   df[target_col] = df[source_col].apply(
    #     lambda x: '|'.join(d['name'] for d in x) if isinstance(x, list) else None
    # )
    # return df
    
df=break_data_points(df)


In [ ]:
df.head()

In [ ]:
break_data_points(df, init_column='production_countries', new_column='cld_production_countries')
break_data_points(df, init_column='production_companies', new_column='cld_production_companies')
break_data_points(df, init_column='spoken_languages', new_column='original_language')


In [ ]:
df['credits'][0]

In [ ]:
import pandas as pd
import ast

# Function to extract cast names
def extract_cast_names(credits):
    return [member['name'] for member in credits.get('cast', [])]

# Function to extract crew names
def extract_crew_names(credits):
    return [member['name'] for member in credits.get('crew', [])]

# Function to extract director
def extract_director(credits):
    for member in credits.get('crew', []):
        if member.get('job') == 'Director':
            return member.get('name')
    return None

# Extract and add new columns
df['cast'] = df['credits'].apply(lambda x: ' | '.join(extract_cast_names(x)))
df['crew'] = df['credits'].apply(lambda x: ' | '.join(extract_crew_names(x)))
df['director'] = df['credits'].apply(extract_director)
df['cast_size'] = df['credits'].apply(lambda x: len(x.get('cast', [])))
df['crew_size'] = df['credits'].apply(lambda x: len(x.get('crew', [])))


In [ ]:
df.head()

### Identify Anomalies with value_count()

In [ ]:
def get_value_counts(df, column):
    """
    Returns value_counts for a single column in a DataFrame.

    Parameters:
    - df (pd.DataFrame): The DataFrame to analyze.
    - column (str): Column name to get value counts for.
    - normalize (bool): If True, returns relative frequencies.
    - dropna (bool): If True, don’t include counts of NaN.

    Returns:
    - pd.Series: Value counts of the specified column.
    """
    if column in df.columns:
        return df[column].value_counts()
    else:
        raise ValueError(f"Column '{column}' not found in DataFrame.")
    
# # def genre_name(df, init_column='genres', new_column='genre_names'):
# # def count_value(df, extrated_column='genre_names'):
# def value_counts_multiple(df, columns):
#     """
#     Returns value_counts() for multiple columns in a DataFrame.

#     Parameters:
#     - df (pd.DataFrame): The DataFrame to analyze.
#     - columns (list): List of column names to apply value_counts() to.

#     Returns:
#     - dict: A dictionary where keys are column names and values are their value_counts().
#     """
#     counts = {}
#     for col in columns:
#         if col in df.columns:
#             counts[col] = df[col].value_counts()
#         else:
#             print(f"Warning: Column '{col}' not found in DataFrame.")
#     return counts
    

In [ ]:
get_value_counts(df, 'genre_names')

In [ ]:
get_value_counts(df, 'cld_production_countries')

In [ ]:
get_value_counts(df, 'original_language')

In [ ]:
def normalize_anomalies(genre_string):
    genres = [g.strip() for g in genre_string.split('|')]
    sorted_genres = sorted(genres)
    return ' | '.join(sorted_genres)

df['genre_names']= df['genre_names'].apply(normalize_anomalies)
df['cld_production_countries']= df['cld_production_countries'].apply(normalize_anomalies)
df['cld_production_companies']= df['cld_production_companies'].apply(normalize_anomalies)
df['original_language']= df['original_language'].apply(normalize_anomalies)



In [ ]:
print(df['genre_names'].value_counts())
print(df['cld_production_countries'].value_counts())
print(df['cld_production_companies'].value_counts())
print(df['original_language'].value_counts())
